# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,6.25,65,40,9.26,CL,1727878433
1,1,margaret river,-33.9500,115.0667,12.01,80,100,12.78,AU,1727878434
2,2,soygaon,20.6000,75.6167,28.00,73,2,1.82,IN,1727878395
3,3,banda aceh,5.5577,95.3222,27.41,81,100,3.34,ID,1727878436
4,4,ribeira grande,38.5167,-28.7000,25.22,83,75,11.83,PT,1727878437


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
plot_humid = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_humid

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head() 

12


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,6.25,65,40,9.26,CL,1727878433
66,66,ilulissat,69.2167,-51.1000,6.01,49,75,7.72,GL,1727878504
162,162,saynshand,44.8824,110.1163,9.27,55,0,4.70,MN,1727878613
204,204,puerto deseado,-47.7503,-65.8938,6.68,55,23,5.80,AR,1727878658
218,218,kzyltu,53.6364,72.3414,7.38,38,0,3.92,KZ,1727878673


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head() 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto natales,CL,-51.7236,-72.4875,65,
66,ilulissat,GL,69.2167,-51.1000,49,
162,saynshand,MN,44.8824,110.1163,55,
204,puerto deseado,AR,-47.7503,-65.8938,55,
218,kzyltu,KZ,53.6364,72.3414,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1} 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}' 

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: Dorotea Patagonia Hostel
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
saynshand - nearest hotel: No hotel found
puerto deseado - nearest hotel: Los Olmos
kzyltu - nearest hotel: Гаухартас
qaqortoq - nearest hotel: Hotel Qaqortoq
lebu - nearest hotel: No hotel found
ulan bator - nearest hotel: LION Hotel
baruun-urt - nearest hotel: No hotel found
tsetserleg - nearest hotel: Буян зочид буудал
las heras - nearest hotel: Posadas del Sur
ornskoldsvik - nearest hotel: Hotell Focus


,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto natales,CL,-51.7236,-72.4875,65,Dorotea Patagonia Hostel
66,ilulissat,GL,69.2167,-51.1000,49,Best Western Plus Hotel Ilulissat
162,saynshand,MN,44.8824,110.1163,55,No hotel found
204,puerto deseado,AR,-47.7503,-65.8938,55,Los Olmos
218,kzyltu,KZ,53.6364,72.3414,38,Гаухартас
256,qaqortoq,GL,60.7167,-46.0333,53,Hotel Qaqortoq
267,lebu,CL,-37.6167,-73.6500,60,No hotel found
303,ulan bator,MN,47.9077,106.8832,49,LION Hotel
356,baruun-urt,MN,46.6806,113.2792,64,No hotel found
458,tsetserleg,MN,47.4750,101.4542,67,Буян зочид буудал


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
plot_map = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)